In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from kerastuner import RandomSearch
from tensorflow.keras.preprocessing.image import ImageDataGenerator

/var/folders/_0/n_t3__qd6k15bv63l5bktb8m0000gn/T/ipykernel_32391/511863004.py:5: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [2]:
import json

# Function to load configurations
def load_config(config_file):
    with open(config_file, 'r') as file:
        config = json.load(file)
    return config

# Load the configuration
config = load_config('config/config.json')

# Access the dataset path
dataset_dir = config['dataset_path']
train_dir = config['train_path']
val_dir = config['val_path']
test_dir = config['test_path']

In [3]:
def build_model(hp):
    model = Sequential([
        Conv2D(
            filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
            kernel_size=hp.Choice('conv_1_kernel', values=[3, 5]),
            activation='relu',
            input_shape=(64, 64, 3)
        ),
        MaxPooling2D(2, 2),
        Conv2D(
            filters=hp.Int('conv_2_filter', min_value=32, max_value=128, step=16),
            kernel_size=hp.Choice('conv_2_kernel', values=[3, 5]),
            activation='relu'
        ),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(
            units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
            activation='relu'
        ),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

In [4]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,  # Adjust the number of trials here
    executions_per_trial=1,
    directory='model_tuning',
    project_name='MushroomClassification'
)

/opt/anaconda3/envs/Thesis/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

Found 194 images belonging to 2 classes.
Found 28 images belonging to 2 classes.


In [6]:
number_of_epochs = 40

In [7]:
tuner.search(
    train_generator,
    epochs=number_of_epochs,
    validation_data=val_generator
)

Trial 10 Complete [00h 03m 27s]
val_accuracy: 0.5

Best val_accuracy So Far: 0.6428571343421936
Total elapsed time: 00h 33m 44s


In [8]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps)

In [9]:
# Train the best model
model.fit(
    train_generator,
    epochs=number_of_epochs,
    validation_data=val_generator
)

Epoch 1/40
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 388ms/step - accuracy: 0.5026 - loss: 0.7089 - val_accuracy: 0.4286 - val_loss: 0.7403
Epoch 2/40
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 218ms/step - accuracy: 0.6049 - loss: 0.6687 - val_accuracy: 0.4286 - val_loss: 0.7427
Epoch 3/40
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 220ms/step - accuracy: 0.5769 - loss: 0.6977 - val_accuracy: 0.4286 - val_loss: 0.7023
Epoch 4/40
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 217ms/step - accuracy: 0.5714 - loss: 0.6933 - val_accuracy: 0.5357 - val_loss: 0.6941
Epoch 5/40
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 231ms/step - accuracy: 0.5515 - loss: 0.6870 - val_accuracy: 0.4643 - val_loss: 0.7045
Epoch 6/40
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 240ms/step - accuracy: 0.6355 - loss: 0.6612 - val_accuracy: 0.4286 - val_loss: 0.7343
Epoch 7/40
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 220ms/step - accuracy: 0.6462 - loss: 0.6609 - val_accuracy: 0.4643 - val_loss: 0.7079
Epoch 8/40
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 217ms/step - accuracy: 0.6055 - loss: 0.6642 - val_accuracy: 0.3929 - val_loss:

In [10]:
loss, accuracy = model.evaluate(val_generator)
print(f"Final validation accuracy: {accuracy*100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 955ms/step - accuracy: 0.5714 - loss: 0.7324
Final validation accuracy: 57.14%


In [11]:
# Save the model, optional
model.save(f'Pth_Files/CNN_model_test_{number_of_epochs}.keras')